In [5]:
import pandas as pd
import plotly_express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import numpy as np

train = pd.read_csv("C:\\Users\\Bigotillos\\ubiqum\\3.- IoT Analytics\\Wifi Locationing\Datasets\\trainingData.csv")
validation = pd.read_csv("C:\\Users\\Bigotillos\\ubiqum\\3.- IoT Analytics\\Wifi Locationing\Datasets\\validationData.csv")

import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

phoneid_brand_no_version = {
    0: "Celkon A27", 1: "GT-I8160", 2: "GT-I8160", 3: "GT-I9100", 4: "GT-I9300", 5: "GT-I9505", 6: "GT-S5360",
    7: "GT-S6500", 8: "Galaxy Nexus", 9: "Galaxy Nexus", 10: "HTC Desire HD", 11: "HTC One", 12: "HTC One",
    13: "HTC Wildfire S", 14: "LT22i", 15: "LT22i", 16: "LT26i", 17: "M1005D", 18: "MT11i", 19: "Nexus 4",
    20: "Nexus 4", 21: "Nexus S", 22: "Orange Monte Carlo", 23: "Transformer TF101", 24: "bq Curie", 25: "Nexus 5",
    26: "Orange Rono", 27: "D2303", 28: "Wildfire S A510e", 29: "GT-I9505"
}

phone_id_android_version = {
    0: "4.0.4(6577)", 1: "2.3.6", 2: "4.1.2", 3: "4.0.4", 4: "4.1.2", 5: "4.2.2", 6: "2.3.6", 7: "2.3.6", 8: "4.2.2",
    9: "4.3", 10: "2.3.5", 11: "4.1.2", 12: "4.2.2", 13: "2.3.5", 14: "4.0.4", 15: "4.1.2", 16: "4.0.4", 17: "4.0.4",
    18: "2.3.4", 19: "4.2.2", 20: "4.3", 21: "4.1.2", 22: "2.3.5", 23: "4.0.3", 24: "4.1.1", 25: "5.0.1", 26: "4.4.2",
    27: "4.4.4", 28: "4.2.2", 29: "4.4.2"
}


def select_predictor_variables(df, name):
    return df.drop(name, axis=1)


def select_target_variable(df, name):
    return df[name]


acc = []
d = {}

anonymized_vars = ["LATITUDE", "LONGITUDE", "FLOOR", "BUILDINGID", "SPACEID", "RELATIVEPOSITION", "USERID", "TIMESTAMP",
                   "PHONEID"]
vars_to_predict = ["LATITUDE", "LONGITUDE", "FLOOR", "BUILDINGID"]

for var in vars_to_predict:
    X_train = select_predictor_variables(train, anonymized_vars)
    X_test = select_predictor_variables(validation, anonymized_vars)
    y_train = select_target_variable(train, var)
    y_test = select_target_variable(validation, var)

    if var in ["LATITUDE", "LONGITUDE"]:
        RFR = RandomForestRegressor(random_state=42)
        RFR_fit = RFR.fit(X_train, y_train.values.ravel())
        predictions = RFR_fit.predict(X_test)

        d[var] = predictions

    else:
        RFC = RandomForestClassifier(random_state=42)
        RFC_fit = RFC.fit(X_train, y_train)
        predictionss = RFC_fit.predict(X_test)

        d[var] = predictionss
        acc.append(accuracy_score(y_test, predictionss))


C:\Users\Bigotillos\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\Bigotillos\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\Bigotillos\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\Bigotillos\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



In [12]:
import plotly.io as pio
#pio.renderers.default = 'notebook_connected'
pio.renderers.default = 'browser'
#pio.renderers.default = 'vscode'

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=d["LONGITUDE"], 
            y=d["LATITUDE"], 
            z=d["FLOOR"],
            mode='markers',
            marker=dict(
                size=3,
                color=(abs(d["LONGITUDE"] - validation["LONGITUDE"]) + abs(d["LATITUDE"] - validation["LATITUDE"]) + 
                       4*abs(d["FLOOR"] - validation["FLOOR"]) + 50*(d["BUILDINGID"] - validation["BUILDINGID"])),
                opacity=0.8,
                cmax=max(abs(d["LONGITUDE"] - validation["LONGITUDE"]) + abs(d["LATITUDE"] - validation["LATITUDE"]) + 
                       4*abs(d["FLOOR"] - validation["FLOOR"]) + 50*(d["BUILDINGID"] - validation["BUILDINGID"])),
                cmin=min(abs(d["LONGITUDE"] - validation["LONGITUDE"]) + abs(d["LATITUDE"] - validation["LATITUDE"]) + 
                       4*abs(d["FLOOR"] - validation["FLOOR"]) + 50*(d["BUILDINGID"] - validation["BUILDINGID"])),
                colorbar=dict(
                    title="Total Error"
                ),
                colorscale="Picnic"
            )
        )
    ]
)

fig.update_layout(scene = dict(
                    xaxis_title='Longitude',
                    yaxis_title='Latitude',
                    zaxis_title='Floor')
                 )
fig.show()

In [34]:
import pandas as pd
import numpy as np
import xgboost as xgb

from pathlib import Path
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

# Selector of variables functions
def select_predictor_variables(df, name):
    return df.drop(name, axis=1)

def select_target_variable(df, name):
    return df[name]

min_transf=-105 
cv_buildingid=False
cv_floor=False
cv_lat_lon=False
knn=True

# CV hyper parameter tuning list. Used it to find the best parameters for the xgboost model. Fixed the values for
# subsequent models.
param_distributions = {
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7],
}

# Path shenanigans
path_handler = {}

# Depending if the code is executed through the console or the IDE the relative path changes.
# This is a workaround that discards the repeated folders.
for word in (str(Path.cwd()) + "\\3.- IoT Analytics\\Wifi Locationing\\Datasets").split("\\"):
    if word in path_handler:
        path_handler[word] += 1
    else:
        path_handler[word] = 1

datasets_path = "\\".join(path_handler.keys())

train = pd.read_csv(datasets_path + "\\trainingData.csv")
validation = pd.read_csv(datasets_path + "\\validationData.csv")

theta = np.radians(29)  # 29 is almost horizontal
c, s = np.cos(theta), np.sin(theta)
r = np.array(((c, -s), (s, c)))

train[["LONGITUDE", "LATITUDE"]] = np.transpose(np.dot(r, np.transpose(train[["LONGITUDE", "LATITUDE"]])))

# Transform 100s values to something more meaningful for the model
if min_transf:
    train = train.apply(lambda x: np.where(x == 100, min_transf, x))
    validation = validation.apply(lambda x: np.where(x == 100, min_transf, x))

anonymized_vars = ["LATITUDE", "LONGITUDE", "FLOOR", "BUILDINGID", "SPACEID",
                   "RELATIVEPOSITION", "USERID", "TIMESTAMP", "PHONEID"]

# Cascade: first BUILDINGID as target variable.
x_train = select_predictor_variables(train, anonymized_vars)
x_test = select_predictor_variables(validation, anonymized_vars)

# Shift all the values in the data set by 105.
x_train = x_train.apply(lambda x: x + 105)
x_test = x_test.apply(lambda x: x + 105)

# Apply l2 row normalization. The sum of each row will be 1. This should help in taking into account the variance
# in height, mobile phone model, etc.
x_train = pd.DataFrame(normalize(X=x_train, norm="l2", axis=1), columns=x_train.columns)
x_test = pd.DataFrame(normalize(X=x_test, norm="l2", axis=1), columns=x_test.columns)

y_train = select_target_variable(train, "BUILDINGID")

if cv_buildingid:
    xgbc_building = RandomizedSearchCV(estimator=xgb.XGBClassifier(verbosity=1),
                                       param_distributions=param_distributions,
                                       scoring="accuracy", verbose=10,
                                       n_jobs=4, random_state=42, cv=5)
elif knn:
    xgbc_building = KNeighborsClassifier(n_neighbors=5)
else:
    xgbc_building = xgb.XGBClassifier(
        max_depth=15, objective="reg:squarederror", min_child_weight=1,
        n_estimators=400, colsample_bytree=0.4, gamma=0.4, learning_rate=0.25,
        nthread=4)

xgbc_fit_building = xgbc_building.fit(x_train, y_train.values.ravel())

# Cascade: use the prediction as a predictor for other target variables.
x_test["BUILDINGID"] = xgbc_fit_building.predict(x_test)
x_train["BUILDINGID"] = train["BUILDINGID"]

n_buildings = train["BUILDINGID"].unique().tolist()

preds_build = pd.DataFrame()

# Once BUILDINGID is predicted, the test set can be subset. Apply a different xgboost to each BUILDINGID.
for build in n_buildings:
    x_train_subset = x_train[x_train["BUILDINGID"] == build]
    y_train_subset = select_target_variable(train[train["BUILDINGID"] == build], "FLOOR")
    x_test_subset = x_test[x_test["BUILDINGID"] == build]

    if cv_floor:
        xgbc_floor = RandomizedSearchCV(estimator=xgb.XGBClassifier(verbosity=1),
                                        param_distributions=param_distributions,
                                        scoring="accuracy", verbose=10,
                                        n_jobs=4, random_state=42, cv=5)
    else:
        xgbc_floor = xgb.XGBClassifier(
            max_depth=15, objective="reg:squarederror", min_child_weight=1,
            n_estimators=400, colsample_bytree=0.4, gamma=0.4, learning_rate=0.25,
            nthread=4)

    xgbr_fit_floor_build = xgbc_floor.fit(x_train_subset, y_train_subset.values.ravel())

    preds_build = preds_build.append(pd.DataFrame(xgbr_fit_floor_build.predict(x_test_subset),
                                                  index=x_test_subset.index))

x_test["FLOOR"] = preds_build
x_train["FLOOR"] = train["FLOOR"]

lat_lon_preds = pd.DataFrame()

# The best result obtained is through knn with k = 5 for latitude and longitude.
for var in ["LATITUDE", "LONGITUDE"]:
    y_train = select_target_variable(train, var)

    if cv_lat_lon:
        xgbr_lat_lon = RandomizedSearchCV(estimator=xgb.XGBRegressor(verbosity=1, objective="reg:squarederror"),
                                          param_distributions=param_distributions,
                                          scoring="neg_mean_absolute_error", verbose=10, n_jobs=4, random_state=42,
                                          cv=5)
    elif knn:
        xgbr_lat_lon = KNeighborsRegressor(n_neighbors=5)
    else:
        xgbr_lat_lon = xgb.XGBRegressor(
            max_depth=15, objective="reg:squarederror", min_child_weight=1,
            n_estimators=400, colsample_bytree=0.4, gamma=0.4, learning_rate=0.25,
            nthread=4)

    xgbr_fit_lat_lon = xgbr_lat_lon.fit(x_train, y_train.values.ravel())
    lat_lon_preds[var] = xgbr_fit_lat_lon.predict(x_test)

validation[["LONGITUDE", "LATITUDE"]] = np.transpose(np.dot(r, np.transpose(validation[["LONGITUDE", "LATITUDE"]])))



In [35]:
distance_75_xgb = np.sqrt((lat_lon_preds["LONGITUDE"] - validation["LONGITUDE"])**2
                              + (lat_lon_preds["LATITUDE"] - validation["LATITUDE"])**2)+ 4*abs(x_test["FLOOR"] - validation["FLOOR"]) + 50*(x_test["BUILDINGID"] != validation["BUILDINGID"])

In [37]:
import plotly.io as pio
#pio.renderers.default = 'notebook_connected'
pio.renderers.default = 'browser'
#pio.renderers.default = 'vscode'

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=validation["LONGITUDE"], 
            y=validation["LATITUDE"], 
            z=validation["FLOOR"],
            mode='markers',
            marker=dict(
                size=3,
                color=(distance_75_xgb),
                opacity=0.8,
                cmax=max(distance_75_xgb),
                cmin=min(distance_75_xgb),
                colorbar=dict(
                    title="distance_75"
                ),
                colorscale="Picnic"
            )
        )
    ]
)

fig.update_layout(scene = dict(
                    xaxis_title='Longitude',
                    yaxis_title='Latitude',
                    zaxis_title='Floor')
                 )
fig.show()